<a href="https://colab.research.google.com/github/SuganyaMurugesan/TrafficLightDetection/blob/master/SmallObject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/HCLHackathon')

In [5]:
"""
Mask R-CNN
Train on the toy Balloon dataset and implement color splash effect.
Copyright (c) 2018 Matterport, Inc.
Licensed under the MIT License (see LICENSE for details)
Written by Waleed Abdulla
------------------------------------------------------------
Usage: import the module (see Jupyter notebooks for examples), or run from
       the command line as such:
    # Train a new model starting from pre-trained COCO weights
    python3 balloon.py train --dataset=/path/to/balloon/dataset --weights=coco
    # Resume training a model that you had trained earlier
    python3 balloon.py train --dataset=/path/to/balloon/dataset --weights=last
    # Train a new model starting from ImageNet weights
    python3 balloon.py train --dataset=/path/to/balloon/dataset --weights=imagenet
    # Apply color splash to an image
    python3 balloon.py splash --weights=/path/to/weights/file.h5 --image=<URL or path to file>
    # Apply color splash to video using the last weights you trained
    python3 balloon.py splash --weights=last --video=<URL or path to file>
"""

import os
import sys
import json
import datetime
import numpy as np
import skimage.draw

# Root directory of the project
ROOT_DIR = os.getcwd()

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

#from samples.balloon import balloon

%matplotlib inline
# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")
TRAFFICLIGHT_DIR = "/content/drive/My Drive/Colab Notebooks/HCLHackathon/data"

Using TensorFlow backend.


In [0]:
class TrafficLightConfig(Config):
    """Configuration for training on the toy  dataset.
    Derives from the base Config class and overrides some values.
  
      """
    # Give the configuration a recognizable name
    NAME = "TrafficLight"

    # Whether to use image augmentation in training mode
    AUGMENT = True

    # Whether to use image scaling and rotations in training mode
    SCALE = True

    # Optimizer, default is 'SGD'
    OPTIMIZER = 'ADAM'

    # Train on 1 GPU and 2 images per GPU.
    GPU_COUNT = 1
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # background + TrafficLight    

    # Backbone encoder architecture
    BACKBONE = 'resnet101'

    # Using smaller anchors because traffic lights  are small
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)

    # How many anchors per image to use for RPN training
    RPN_TRAIN_ANCHORS_PER_IMAGE = 320  #

    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 2048
    POST_NMS_ROIS_INFERENCE = 2048

    # Number of ROIs per image to feed to classifier/mask heads
    TRAIN_ROIS_PER_IMAGE = 512
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more proposals.
    RPN_NMS_THRESHOLD = 0.7
    # Maximum number of ground truth instances to use in one image
    MAX_GT_INSTANCES = 256

    # Max number of final detections
    DETECTION_MAX_INSTANCES = 400

    # Minimum probability value to accept a detected instance
    # ROIs below this threshold are skipped
    DETECTION_MIN_CONFIDENCE = 0.75

    # Non-maximum suppression threshold for detection
    DETECTION_NMS_THRESHOLD = 0.3  # 0.3

    # Threshold number for mask binarization, only used in inference mode
    DETECTION_MASK_THRESHOLD = 0.35


In [0]:
# ############################################################
# #  Dataset
# ############################################################

# class TrafficLightDataset(utils.Dataset):

#     def load_trafficlight(self, dataset_dir, subset):
#         """Load a subset of the Balloon dataset.
#         dataset_dir: Root directory of the dataset.
#         subset: Subset to load: train or val
#         """
#         # Add classes. We have only one class to add.
#         self.add_class("trafficlight", 1, "trafficlight")

#         # Train or validation dataset?
#         assert subset in ["train", "val"]
#         dataset_dir = os.path.join(dataset_dir, subset)
#         annotations = json.load(open(os.path.join(dataset_dir, "via_region.json")))
        
#         # Add images
#         for a in annotations:
#           rect = a["boxes"]
#           image_id = a["path"].split('/')[4]
#           image_path = os.path.join(dataset_dir, image_id)
#           image = skimage.io.imread(image_path)
#           height, width = image.shape[:2]
#           self.add_image(
#                 "trafficlight",
#                 image_id=image_id,  # use file name as a unique image id
#                 path=image_path,
#                 width=width, height=height,
#                 rect=rect)
          
#     def load_mask(self, image_id):
#         """Generate instance masks for an image.
#        Returns:
#         masks: A bool array of shape [height, width, instance count] with
#             one mask per instance.
#         class_ids: a 1D array of class IDs of the instance masks.
#         """
#         # If not a trafficlight dataset image, delegate to parent class.
#         image_info = self.image_info[image_id]
#         if image_info["source"] != "trafficlight":
#             return super(self.__class__, self).load_mask(image_id)

#         # Convert rect to a bitmap mask of shape
#         # [height, width, instance_count]
#         if image_info["source"] == "trafficlight":
#           info = self.image_info[image_id]
#           print(info)
#           mask = np.zeros([info["height"], info["width"], len(info["rect"])],dtype=np.uint8)
#           print(type(mask)) 
#         for i, p in enumerate(info["rect"]):
#             # Get indexes of pixels inside the rect and set them to 1
            
#             start = (p["y_min"], p["x_min"])
#             end = (p["y_max"], p["x_max"])
#             print("START:", start)
#             print("END:", end)
#             rr, cc = skimage.draw.rectangle(start, end)
#             rr = rr.astype(int)
#             cc = cc.astype(int)
#             mask[rr, cc, i] = 1

#         return mask.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32)

#     def image_reference(self, image_id):
#         """Return the path of the image."""
#         print("IN IMAGE REFERENCE ***************$$$$$$$$")
#         info = self.image_info[image_id]
#         if info["source"] == "trafficlight":
#             return info["path"]
#         else:
#             super(self.__class__, self).image_reference(image_id)


In [0]:
############################################################
#  Dataset
############################################################

class TrafficLightDataset(utils.Dataset):

    def load_trafficlight(self, df, train_path):
        """Load a subset of the Balloon dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have only one class to add.
        self.add_class("trafficlight", 1, "trafficlight")

        # Train or validation dataset
        #image_path = df['path'].split('/')[4]
        #image = skimage.io.imread(image_path)
        #height, width = image.shape[:2]
        #print(image_path)
        print(df.index)
        for ind in df.index: 
          image_path = os.path.join(train_path, ind)
          image = skimage.io.imread(image_path)
          height, width = image.shape[:2]
          self.add_image(
                "trafficlight",
                image_id=ind,  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                rect=df['rect'][ind])

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a trafficlight dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "trafficlight":
            return super(self.__class__, self).load_mask(image_id)

        # Convert rect to a bitmap mask of shape
        # [height, width, instance_count]
        if image_info["source"] == "trafficlight":
          info = self.image_info[image_id]
          mask = np.zeros([info["height"], info["width"], len(info["rect"])],dtype=np.uint8)
          
        for i, p in enumerate(info["rect"]):
            # Get indexes of pixels inside the rect and set them to 1
            
            start = (p["y_min"], p["x_min"])
            end = (p["y_max"], p["x_max"])
            rr, cc = skimage.draw.rectangle(start, end)
            rr = rr.astype(int)
            cc = cc.astype(int)
            mask[rr, cc, i] = 1

        return mask.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        print("IN IMAGE REFERENCE ***************$$$$$$$$")
        info = self.image_info[image_id]
        if info["source"] == "trafficlight":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)


In [0]:
def train_validation_split(train_path, seed=10, test_size=0.1):

    """
    Split the dataset into train and validation sets.
    External data and mosaics are directly appended to training set.
    """
    from sklearn.model_selection import train_test_split
    import pandas as pd
    import os
    df = pd.read_json(os.path.join(train_path, "via_region.json"))
    print(df)
    df['filename'] = df['path'].apply(lambda x: x.split('/')[4])
    df['rect'] = df['boxes']
    
    df = df.set_index('filename')
    train_list, val_list = train_test_split(df, test_size=test_size,
                                            random_state=seed)
    return train_list, val_list

In [0]:
1
6203
36230.

if __name__ == '__main__':
  import time
  start = time.time()

  train_list, val_list = train_validation_split("/content/drive/My Drive/Colab Notebooks/HCLHackathon/RIBDATA/full", seed=11, test_size=0.1)
# Training dataset.
  dataset_train = TrafficLightDataset()
  dataset_train.load_trafficlight(train_list, "/content/drive/My Drive/Colab Notebooks/HCLHackathon/RIBDATA/full")
  dataset_train.prepare()

# Validation dataset
  dataset_val = TrafficLightDataset()
  dataset_val.load_trafficlight(val_list, "/content/drive/My Drive/Colab Notebooks/HCLHackathon/RIBDATA/full")
  dataset_val.prepare()

# Create model configuration in training mode
  config = TrafficLightConfig()
  config.STEPS_PER_EPOCH = len(train_list)//config.BATCH_SIZE
  config.VALIDATION_STEPS = len(val_list)//config.BATCH_SIZE
  config.display()

  model = modellib.MaskRCNN(mode="training", config=config,
                              model_dir=DEFAULT_LOGS_DIR)

# Model weights to start training with
  init_with = "pretrained"  # imagenet, last, or some pretrained model

  if init_with == "imagenet":
    weights_path = "/content/drive/My Drive/Colab Notebooks/HCLHackathon/logs/trafficlight20191128T1144/mask_rcnn_trafficlight_0050.h5"
    model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])

  elif init_with == "last":
  # Load the last model you trained and continue training
    weights_path = "/content/drive/My Drive/Colab Notebooks/HCLHackathon/logs/trafficlight20191128T1144/mask_rcnn_trafficlight_0050.h5"
    print('Loading weights from ', weights_path)
    model.load_weights(weights_path, by_name=True)
  elif init_with == 'pretrained':
    weights_path = '/content/drive/My Drive/Colab Notebooks/HCLHackathon/logs/trafficlight20191128T1144/mask_rcnn_trafficlight_0075.h5'
    model.load_weights(weights_path, by_name=True)
  print('Loading weights from ', weights_path)

  # Train the model for 75 epochs

  #model.train(dataset_train, dataset_val,
                # learning_rate=1e-4,
                # epochs=25,
                # layers='all')

  # model.train(dataset_train, dataset_val,
                # learning_rate=1e-5,
                # epochs=50,
                # layers='all')

  model.train(dataset_train, dataset_val,
                learning_rate=1e-6,
                epochs=100,
                layers='all')

  print('Elapsed time', round((time.time() - start)/60, 1), 'minutes')

                                                 boxes                                               path
0                                                   []  ./rgb/additional/2015-10-05-10-52-01_bag/24594...
1                                                   []  ./rgb/additional/2015-10-05-10-52-01_bag/24664...
2                                                   []  ./rgb/additional/2015-10-05-10-52-01_bag/24734...
3    [{'label': 'Yellow', 'occluded': False, 'x_max...  ./rgb/additional/2015-10-05-10-55-33_bag/56988...
4    [{'label': 'Red', 'occluded': False, 'x_max': ...  ./rgb/additional/2015-10-05-10-55-33_bag/57058...
..                                                 ...                                                ...
210  [{'label': 'Red', 'occluded': False, 'x_max': ...  ./rgb/additional/2015-10-05-16-02-30_bag/72051...
211  [{'label': 'GreenStraight', 'occluded': False,...  ./rgb/additional/2015-10-05-16-02-30_bag/72058...
212  [{'label': 'Green', 'occluded': False, 'x

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


/usr/local/lib/python3.6/dist-packages/keras/engine/training_generator.py:49: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'




Epoch 76/100
